In [1]:
# importing the necessary libraries and the MNIST dataset
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.mnist import load_data

In [2]:
train_data, test_data = load_data()

In [15]:
# defining functions for the two networks.
# Both the networks have two hidden layers
# and an output layer which are densely or 
# fully connected layers defining the 
# Generator network function
def generator(z, reuse = None):
    with tf.compat.v1.variable_scope('gen', reuse = reuse):
        hidden1 = tf.keras.layers.Dense(units = 128, 
                            activation = tf.nn.leaky_relu)(z)
                              
        hidden2 = tf.keras.layers.Dense(units = 128, activation = tf.nn.leaky_relu)(hidden1)
             
        output = tf.keras.layers.Dense(units = 784, activation = tf.nn.tanh)(hidden2)
          
        return output

# defining the Discriminator network function 
def discriminator(X, reuse = None):
    with tf.compat.v1.variable_scope('dis', reuse = reuse):
        hidden1 = tf.keras.layers.Dense(units = 128,
                            activation = tf.nn.leaky_relu)(X)
                              
        hidden2 = tf.keras.layers.Dense(units = 128, activation = tf.nn.leaky_relu)(hidden1)
                 
        logits = tf.keras.layers.Dense(units = 1)(hidden2)
        output = tf.sigmoid(logits)
          
        return output, logits

In [16]:
# creating placeholders for the outputs
tf.compat.v1.reset_default_graph()

tf.compat.v1.disable_eager_execution()
  
real_images = tf.compat.v1.placeholder(tf.float32, shape =[None, 784])
z = tf.compat.v1.placeholder(tf.float32, shape =[None, 100])
  
G = generator(z)
D_output_real, D_logits_real = discriminator(real_images)
D_output_fake, D_logits_fake = discriminator(G, reuse = True)

In [17]:
# defining the loss function
def loss_func(logits_in, labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                          logits = logits_in, labels = labels_in))

In [18]:
 # Smoothing for generalization
D_real_loss = loss_func(D_logits_real, tf.ones_like(D_logits_real)*0.9)
D_fake_loss = loss_func(D_logits_fake, tf.zeros_like(D_logits_real))
D_loss = D_real_loss + D_fake_loss
  
G_loss = loss_func(D_logits_fake, tf.ones_like(D_logits_fake))

In [25]:
lr = 0.001 #learning rate alpha

In [26]:
tvars = tf.compat.v1.trainable_variables() 
d_vars =[var for var in tvars if 'dis' in var.name]
g_vars =[var for var in tvars if 'gen' in var.name]

In [27]:
D_trainer = tf.compat.v1.train.AdamOptimizer(lr).minimize(D_loss, var_list = d_vars)
G_trainer = tf.compat.v1.train.AdamOptimizer(lr).minimize(G_loss, var_list = g_vars)

In [29]:
batch_size = 100 # batch size
epochs = 500 # number of epochs. The higher the better the result
init = tf.compat.v1.global_variables_initializer()

In [44]:
# creating a session to train the networks
samples =[] # generator examples
  
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        num_batches = len(train_data[0])//batch_size
          
        for i in range(num_batches):
            batch = train_data[0][i*batch_size : (i+1)*batch_size]
            batch_images = batch.reshape((batch_size, 784))
            batch_images = batch_images * 2-1
            batch_z = np.random.uniform(-1, 1, size =(batch_size, 100))
            _= sess.run(D_trainer, feed_dict ={real_images:batch_images, z:batch_z})
            _= sess.run(G_trainer, feed_dict ={z:batch_z})
              
        print("on epoch{}".format(epoch))
          
        sample_z = np.random.uniform(-1, 1, size =(1, 100))
        gen_sample = sess.run(generator(z, reuse = True),
                                 feed_dict ={z:sample_z})
          
        samples.append(gen_sample)

on epoch0


FailedPreconditionError: Graph execution error:

Detected at node 'gen_2/dense_13/BiasAdd/ReadVariableOp' defined at (most recent call last):
    File "C:\Users\TEJAS\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\TEJAS\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\TEJAS\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\TEJAS\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\TEJAS\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\TEJAS\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\TEJAS\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      s